# ABS Quarterly Lending 5601

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd
import readabs as ra
from readabs import metacol as mc
from mgplot import (
    line_plot_finalise,
    seastrend_plot_finalise,
    multi_start,
)

# local imports
from abs_helper import get_abs_data

# pandas display settings
pd.options.display.max_rows = 9999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, _ = get_abs_data("5601.0")

## Plotting

### Set-up

### Headline Indicators

In [3]:
def fix_title(title, lfooter):
    """Title beautification."""

    check = [
        "Total housing excluding refinancing",
        "Total purpose excluding refinancing",
        "New loan commitments",
        "Fixed term loans",
        "Value",
    ]
    for c in check:
        text = f"{c} ;"
        if text in title:
            title = title.replace(text, "")
            lfooter = lfooter + f"{c}. "
    title = (
        title.replace("Businesses", "")
        .replace("Business Finance", "Business Finance -")
        .replace("Households", "Households -")
        .replace(";", "")
        .replace("    ", " ")
        .replace("   ", " ")
        .replace("  ", " ")
        .strip()
    )
    return title, lfooter

In [4]:
def headline_charts() -> None:
    """Produce headline charts."""

    rows = [
        "Households ;  Housing Finance ;  Total dwellings excluding refinancing ;  New loan commitments ;  Value ;",
        "Businesses ;  Business Finance ;  Fixed term loans ;  Purchase of property ;  New loan commitments ;  Value ;",
        "Businesses ;  Business Finance ;  Fixed term loans ;  Construction ;  New loan commitments ;  Value ;",
    ]

    for headline in rows:
        # get data
        f = pd.DataFrame()
        for series_type in "Seasonally Adjusted", "Trend":
            found = meta[
                meta[mc.stype].str.contains(series_type)
                & meta[mc.did].str.contains(headline)
            ]
            if len(found) != 1:
                print(
                    f"Error: {len(found)} rows found for {series_type} {headline}"
                )
                continue
            row = found.iloc[0]
            series_id, units, table, did = (
                row[mc.id],
                row[mc.unit],
                row[mc.table],
                row[mc.did],
            )
            f[series_type] = (abs_dict[table])[series_id]
        f_, units = ra.recalibrate(f, units)

        # plot
        title, lfooter = fix_title(did, "Australia. ")
        multi_start(
            pd.DataFrame(f_),  # mypy
            function=seastrend_plot_finalise,
            starts=(0,-17),
            title=title,
            ylabel=f"{units} / {f_.index.freqstr[0]}",
            lfooter=lfooter,
            rfooter=f"{source} T{table}",
            show=SHOW,
        )


headline_charts()

### Business

In [5]:
def business_charts() -> None:
    """Produce business charts."""

    rows = meta[
        meta[mc.did].str.contains("Business")
        & meta[mc.did].str.contains("loans")
        & meta[mc.did].str.contains("Total purpose")
    ]
    for _, row in rows.iterrows():
        series_id, units, table, did, series_type = (
            row[mc.id],
            row[mc.unit],
            row[mc.table],
            row[mc.did],
            row[mc.stype],
        )
        series, units = ra.recalibrate(abs_dict[table][series_id], units)
        title, lfooter = fix_title(did, f"Australia. {series_type} series. ")
        title = title.replace("Businesses", "").strip()
        line_plot_finalise(
            series,
            title=title,
            ylabel=f"{units} / {series.index.freqstr[0]}",
            lfooter=lfooter,
            rfooter=f"{source} T{table}",
            show=SHOW,
        )


business_charts()

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda


Last updated: 2025-09-04 08:23:00

Python implementation: CPython
Python version       : 3.13.6
IPython version      : 9.4.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

matplotlib: 3.10.5
mgplot    : 0.2.12
pandas    : 2.3.1
readabs   : 0.1.4

Watermark: 2.5.0



In [7]:
print("Done")

Done
